In [1]:
from utils import time_op

In [2]:
# tests the model counting capabilities of SharpSAT-TD and Buddy
from formulas import GPMC, Formula, Buddy, random_k_cnf

n,m,k = 8, 10, 7
cnf, formula = random_k_cnf(n,m,k)
vars = [f"x{idx}" for idx in range(1, n+1)]

solver = GPMC()
satcount_gpmc = time_op(solver.satcount)(cnf)
print(f"GPMC satcount = {satcount_gpmc}")

with Buddy(vars) as model:
    f = time_op(model.node)(formula)
    satcount_bdd = time_op(lambda : f.satcount)()
    print(f"BDD satcount = {satcount_bdd}")


[00005.8749 ms / 0.0059 s / 0.0001 min] satcount
GPMC satcount = 236
[00077.6556 ms / 0.0777 s / 0.0013 min] node
[00000.0045 ms / 0.0000 s / 0.0000 min] <lambda>
BDD satcount = 236.0


In [3]:
n,clauses_to_variables,k = 20, 4.5, 7
m = int(n*clauses_to_variables)
cnf, formula = random_k_cnf(n,m,k)
vars = [f"x{idx+1}" for idx in range(n)]

In [4]:
import importance_measures.bdds as bdd 
from formulas import Buddy

with Buddy(vars) as model:
    x = "x1"
    f = (time_op)(model.node)(formula)
    print(f"bdd size = {f.nodecount} with satcount = {f.satcount}")
    blame, ub = time_op(bdd.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=True)
    infl = time_op(bdd.influence)(f, x)
    print(f"influence {infl:.5f}, {blame:.5f} <= blame <= {ub:.5f}, blame diff {ub-blame:.5f}")

[01712.8141 ms / 1.7128 s / 0.0285 min] node
bdd size = 9828 with satcount = 580920.0
=== COMPUTING BLAME for x1 in BDD with size 9828 ===
k=0 size ell=4763 d result=0.1235 max increase possible=0.4382 
k=1 size ell=31491 d result=0.1537 max increase possible=0.1897 
k=2 size ell=32787 d result=0.1359 max increase possible=0.0403 
k=3 size ell=7144 d result=0.0369 max increase possible=0.0027 
k=4 size ell=466 d result=0.0027 max increase possible=0.0000 
stopped earlier because cannot improve above cutoff.
current value: 0.4528, can be increased by 0.0000 <= 0.0001.
=== DONE ===
[00525.5787 ms / 0.5256 s / 0.0088 min] blame
[00000.4537 ms / 0.0005 s / 0.0000 min] influence
influence 0.12352, 0.45277 <= blame <= 0.45280, blame diff 0.00003


In [5]:
import importance_measures.mc as mc
from formulas import GPMC, Formula

f = time_op(Formula.parse)(formula)
x = "x1"
solver = GPMC(cs=20000)
infl = (time_op)(mc.influence)(f, x, solver)
blame, ub = time_op(mc.blame)(f, x, solver, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=True)
print(f"influence {infl:.5f}, {blame:.5f} <= blame <= {ub:.5f}, blame diff {ub-blame:.5f}")

[01974.0794 ms / 1.9741 s / 0.0329 min] parse
[00345.5582 ms / 0.3456 s / 0.0058 min] influence
=== COMPUTING BLAME for x1 in Formula with size 3843 ===
k=0 size of cnf: 3884 d result=0.1235 max increase possible=0.4382 
k=1 size of cnf: 3955 d result=0.1537 max increase possible=0.1897 
k=2 size of cnf: 4022 d result=0.1359 max increase possible=0.0403 
k=3 size of cnf: 4085 

In [6]:
import importance_measures.mc as mc 
from formulas import Formula, GPMC

f,S = mc.at_most(2, {"x1","x2","x3", "x4"})
f,S = mc.at_most_cnf(3, {1,2,3,4}, 5)
print(f, S)
GPMC().satcount( f )

[[7, -1], [-7, 1], [2, 7, -5], [-2, 5], [-7, 5], [3, 5, -11], [-3, 11], [-5, 11], [4, 11, -9], [-4, 9], [-11, 9], [8, -2, -7], [-8, 7], [-8, 2], [-6, 5, 8], [-6, 3, 8], [6, -3, -5], [6, -8], [-14, 11, 6], [-14, 4, 6], [14, -4, -11], [14, -6], [13, -3, -8], [-13, 8], [-13, 3], [-10, 6, 13], [-10, 4, 13], [10, -4, -6], [10, -13], [12, -4, -13], [-12, 13], [-12, 4], [-12]] {5, 6, 7, 8, 9, 10, 11, 12, 13, 14}


15